# Notebook to compare Penn CHIME fits against all parameter fits

## Imports

In [1]:
from os import environ
from datetime import date

from pandas import DataFrame
from numpy import zeros

from penn_chime.parameters import Parameters, Disposition
from penn_chime.models import (
    SimSirModel,
    sim_sir,
    calculate_admits,
    calculate_dispositions,
)
import penn_chime

from models import sir_step, sihr_step, one_minus_logistic_fcn, FitFcn

In [2]:
COLS_TO_COMPARE = ["susceptible", "infected", "recovered", "hospitalized_new"]

## Set up Penn CHIME model

In [3]:
p = Parameters(
    current_hospitalized=69,
    date_first_hospitalized=date(2020, 3, 7),
    doubling_time=4.0,
    hospitalized=Disposition.create(days=7, rate=0.025),
    icu=Disposition.create(days=9, rate=0.0075),
    infectious_days=14,
    market_share=0.15,
    n_days=100,
    population=3600000,
    recovered=0,
    relative_contact_rate=0.3,
    ventilated=Disposition.create(days=10, rate=0.005),
)

p.doubling_time = None
simsir = SimSirModel(p)

2020-04-09 16:58:10,824 - penn_chime.models - INFO - Using date_first_hospitalized: 2020-03-07; current_date: 2020-04-09; i_day: 33, current_hospitalized: 69
2020-04-09 16:58:10,862 - penn_chime.models - INFO - Estimated doubling_time: 5.266555601832569
2020-04-09 16:58:10,869 - penn_chime.models - INFO - len(np.arange(-i_day, n_days+1)): 134
2020-04-09 16:58:10,870 - penn_chime.models - INFO - len(raw_df): 134


## Tests

### Check that model agrees with Penn CHIME if no policies are in place

Calculate S, I, H, R for no policies

In [4]:
n_days = simsir.raw_df.day.max() - simsir.raw_df.day.min() + 1

policies = [(simsir.beta, n_days)]
raw = sim_sir(
    simsir.susceptible,
    simsir.infected,
    p.recovered,
    simsir.gamma,
    -simsir.i_day,
    policies,
)


calculate_dispositions(raw, simsir.rates, market_share=p.market_share)
calculate_admits(raw, simsir.rates)

raw_df = DataFrame(raw)

day0 = raw_df.iloc[0].fillna(0)

raw_df.head()

,day,susceptible,infected,recovered,ever_infected,ever_hospitalized,hospitalized,ever_icu,icu,ever_ventilated,ventilated,admits_hospitalized,admits_icu,admits_ventilated
0,-33,3.569456e+06,20307.339913,0.000000,20307.339913,76.152525,NaN,22.845757,NaN,15.230505,NaN,NaN,NaN,NaN
1,-32,3.565185e+06,23127.681000,1450.524280,24578.205280,92.168270,16.015745,27.650481,4.804724,18.433654,3.203149,16.015745,4.804724,3.203149
2,-31,3.560327e+06,26333.899264,3102.501494,29436.400757,110.386503,18.218233,33.115951,5.465470,22.077301,3.643647,18.218233,5.465470,3.643647
3,-30,3.554802e+06,29977.061514,4983.494298,34960.555812,131.102084,20.715581,39.330625,6.214674,26.220417,4.143116,20.715581,6.214674,4.143116
4,-29,3.548524e+06,34114.479914,7124.712978,41239.192892,154.646973,23.544889,46.394092,7.063467,30.929395,4.708978,23.544889,7.063467,4.708978


Compute values using new fit function

In [5]:
pars = {
    "beta_i": simsir.beta,
    "gamma_i": simsir.gamma,
    "initial_susceptible": day0.susceptible,
    "initial_infected": day0.infected,
    "initial_hospitalized": day0.hospitalized,
    "initial_recovered": day0.recovered,
    "hospitalization_rate": simsir.rates["hospitalized"] * p.market_share,
}
x = {
    "n_iter": raw_df.shape[0],
}


f = FitFcn(sir_step)
y = f(x, pars)

Check that difference is consistent with zero

In [6]:
pars

{'beta_i': 5.891974409571809e-08,
 'gamma_i': 0.07142857142857142,
 'initial_susceptible': 3569455.6548977033,
 'initial_infected': 20307.339913103864,
 'initial_hospitalized': 0.0,
 'initial_recovered': 0.0,
 'hospitalization_rate': 0.00375}

In [7]:
diff = (raw_df.rename(columns={"hospitalized": "hospitalized_new"}) - y)[COLS_TO_COMPARE]
mean = diff.mean()
sdev = diff.std()
assert (mean.abs() < 2 * sdev).all()
DataFrame([mean, sdev], index=["mean", "sdev"]).T

,mean,sdev
susceptible,1.825996e-10,9.702123e-10
infected,-3.610492e-10,6.822826e-10
recovered,-4.160837e-09,2.794894e-09
hospitalized_new,-1.039591e-13,1.815111e-12


### Check that model agrees with Penn CHIME if no policies are in place

Now compare against Penn CHIME with active social distancing policies.

My fit function takes a function which returns an array of betas, this is implemented below

In [8]:
POLICIES = simsir.gen_policy(p)


def beta_i_fcn(n_iter, **kwargs):
    out = zeros(len(n_iter))
    ii = 0
    for beta, n_days in POLICIES:
        for _ in range(n_days):
            out[ii] = beta
            ii += 1

    return out

In [9]:
day0 = simsir.raw_df.iloc[0]

pars = {
    "beta_i": simsir.beta,
    "gamma_i": simsir.gamma,
    "initial_susceptible": day0.susceptible,
    "initial_infected": day0.infected,
    "initial_hospitalized": day0.hospitalized,
    "initial_recovered": day0.recovered,
    "hospitalization_rate": simsir.rates["hospitalized"] * p.market_share,
}
x = {
    "n_iter": simsir.raw_df.shape[0],
}


f = FitFcn(
    sir_step,
    beta_i_fcn=beta_i_fcn,
)
y = f(x, pars)

In [10]:
diff = (simsir.raw_df.rename(columns={"hospitalized": "hospitalized_new"}) - y)[COLS_TO_COMPARE]
mean = diff.mean()
sdev = diff.std()
assert (mean.abs() < 2 * sdev).all()
DataFrame([mean, sdev], index=["mean", "sdev"]).T

,mean,sdev
susceptible,9.947429e-10,4.857298e-09
infected,-1.633832e-09,2.051451e-09
recovered,-5.516107e-09,6.491375e-09
hospitalized_new,-3.956628e-13,9.732858e-13
